In [1]:
import pandas as pd
import glob
import numpy as np
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from datetime import datetime
from dateutil import tz, parser
import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
# path = r'C:\Users\shash\Downloads\Green_Hackathon-Fall2020-main\Green_Hackathon-Fall2020-main\data\Analysis' # use your path
path = r'C:\Users\umama\shravya python assignment\Analysis'
all_files = glob.glob(path + "/*.csv")
all_files_dict={}

for csv in all_files:
    meter_name=os.path.basename(csv).replace('_results.csv','')
    all_files_dict[meter_name]=os.path.abspath(csv)


In [3]:
labels = pd.read_csv(r'C:\Users\umama\shravya python assignment\Meter Names and Labels.csv')
labels = labels.rename(columns={"Name": "Building_Name"})
labels['Building_Name'] = labels['Building_Name'].str.replace("'" , " ")
labels['Building_Name'] = labels['Building_Name'].str.replace('-' , "")
labels['Building_Name'] = labels['Building_Name'].str.replace('"Spencer Hall"' , '')
labels['Building_Name'] = labels['Building_Name'].str.replace('_kWh' , "")
labels['Building_Name'] = labels['Building_Name'].str.strip()
labels
# print(labels.Building_Name.unique())

,Building_Name,Label
0,GoveSiteLighting,Gove Student Health Center (016) - Site Lights...
1,BaseballFieldSupportBldg,Recreational Field Support Building (257) - Ma...
2,BaseballStadiumPrkLights,Baseball Pavillion (248) -Lighting Submeter
3,BaseballStadiumRecFieldLights,Recreational Field Track & Sports Area (260) -...
4,BaseballStadiumWellPump,Baseball Stadium (177) - Well Pump Submeter
...,...,...
76,Baseball,Baseball Stadium (177) - Main Meter
77,SRC_2H3A,Kaplan Center for Wellness (308) - Main Gym Li...
78,SRC_2H4A,Kaplan Center for Wellness (308) -Fitness Area...
79,SRC_2L1B,Kaplan Center for Wellness (308) - Track Light...


In [4]:
building_options=[]
for building_name in list(labels.Building_Name.unique()):
    building_options.append({'label':building_name,'value':building_name})
    

In [5]:
app = dash.Dash(__name__)
# App layout
app.layout = html.Div([
    
    html.H1("UNCG ENERGY CONSUMPTION DASHBOARD", style={'text-align': 'center'}),
    
    html.Label("CONSUMPTION", style={'text-align': 'center'}),
    
    dcc.RadioItems(id='consumption',
        options=[
            {'label': 'Total', 'value': 'Total'},
            {'label': 'Average', 'value': 'Average'}
        ],
        value='Total'
    ),
    
    html.Label("Frequency", style={'text-align': 'center'}),
    
    dcc.RadioItems(id='freq',
        options=[
            {'label': 'Hourly', 'value': 'hourly'},
            {'label': 'Daily',  'value': 'daily'},
            {'label': 'Weekly', 'value': 'weekly'},
            {'label': 'Monthly','value': 'monthly'}
        ],
        value='monthly'
    ),
    
    dcc.Checklist(id='check',
    options=[
        {'label': 'Actual', 'value': 'Actual'},
        {'label': 'Predicted', 'value': 'Predicted'}
    ],
    value=['Actual']),
    
    dcc.Dropdown(
        id='meters',
        options=building_options,
        value='GoveSiteLighting',
        multi=True),
    
    
    html.Div(id='dd-output-container'),
    
    html.Div(id='output_container', children=[]),
    
    html.Br(),

    dcc.Graph(id='predicted_graph', figure={})

])


In [6]:
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='predicted_graph', component_property='figure')],
    [Input(component_id='consumption', component_property='value'),
    Input(component_id='freq', component_property='value'),
    Input(component_id='check', component_property='value'),
    Input(component_id='meters', component_property='value')]
)
def update_graph(cons_agg,freq,act_pred,meters):

    container = "The chosen values by user are: {}".format(freq)


    def get_data(cons_agg,freq,act_pred,meter):
        
        cols = ['Actual','Predicted','obs_ci_lower','obs_ci_upper','Datetime']
        file_path=all_files_dict[meter]
        csv_file = open(file_path)
        dff = pd.read_csv(csv_file ,encoding = "ISO-8859-1",engine = 'python', sep=',', header=0,names=cols,error_bad_lines=False)
        dff['Datetime'] = pd.to_datetime(dff['Datetime'],utc=True,infer_datetime_format=False)

        
        if freq=='daily':
            if cons_agg == 'Total':
                dff1 = dff.groupby(pd.Grouper(key='Datetime', freq='D'))['Actual','Predicted'].sum().reset_index()
            else:
                dff1 = dff.groupby(pd.Grouper(key='Datetime', freq='D'))['Actual','Predicted'].mean().reset_index()

        elif freq=='weekly':
            if cons_agg == 'Total':
                dff1 = dff.groupby(pd.Grouper(key='Datetime', freq='W'))['Actual','Predicted'].sum().reset_index()
            else:
                dff1 = dff.groupby(pd.Grouper(key='Datetime', freq='W'))['Actual','Predicted'].mean().reset_index()
        elif freq=='monthly':
            if cons_agg == 'Total':
                dff1=dff.groupby(pd.Grouper(key='Datetime',freq='M'))['Actual','Predicted'].sum().reset_index()
            else:
                dff1 = dff.groupby(pd.Grouper(key='Datetime', freq='M'))['Actual','Predicted'].mean().reset_index()
        else:
            if cons_agg == 'Total':
                dff1 = dff
            else:
                dff1 = dff
        csv_file.close()
        return(dff1)

    
    if type(meters)==str:
        meters=[meters]
    traces=[]
    for meter in meters:        
        data=get_data(cons_agg,freq,act_pred,meter)
        if 'Actual' in act_pred:     
            traces.append(go.Scatter(x=data['Datetime'], y=data['Actual'],mode = 'lines+markers',name = 'line+markers'))
#             fig = px.scatter(x=data['Datetime'], y=data['Actual'])
        if 'Predicted' in act_pred:
            traces.append(go.Scatter(x=data['Datetime'], y=data['Predicted'],mode = 'lines+markers',name = 'line+markers'))

#             fig.add_trace(go.Scatter(x=data['Datetime'], y=data['Predicted'], mode='lines',name='Predicted'))
    fig=go.Figure(data=traces)


    return container, fig



In [7]:
if __name__ == '__main__':
#     app.run_server(debug=True)
    app.run_server(debug=False,host = '127.0.0.1')
    
    
    

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Nov/2020 23:39:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2020 23:39:42] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2020 23:39:42] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2020 23:39:43] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2020 23:39:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2020 23:39:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2020 23:40:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2020 23:40:17] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2020 23:40:21] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2020 23:40:26] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2020 23:40:39] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Nov/2020 23:40:48] "POST /_dash-update-component